In [ ]:
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel,AsyncOpenAI, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from IPython.display import Markdown, display

In [ ]:
model = OpenAIChatCompletionsModel(model="gpt-oss:20b",
                                  openai_client=AsyncOpenAI(base_url="http://localhost:11434/v1",api_key="ollama"))

In [ ]:
# Create the agent
agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant",
    model=model
)

In [ ]:
result = await Runner.run(starting_agent=agent,input="Who is the father of Mahadev? Answer in short.")
display(Markdown(result.final_output))

OPENAI_API_KEY is not set, skipping trace export


Mahadev (Shiva) is considered a self‑existing, parentless deity – he has no father.

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is no

In [ ]:
result = Runner.run_streamed(agent, "I am from India. Create a meal plan for a week. Answer in short.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

**Week‑long Indian Meal Plan (short version)**  

| Day | Breakfast | Lunch | Dinner |
|-----|-----------|-------|--------|
| Mon | Poha with peas & peanuts | Chole (chickpea curry) + 2 phulkas | Palak Paneer + steamed rice |
| Tue | Idli + coconut‑masala chutney | Vegetable Biryani + raita | Dal Makhani + roti |
| Wed | Upma with carrots & beans | Rajma + jeera rice | Aloo Gobi + paratha |
| Thu | Dalia (broken‑wheat) with milk | Tofu Masala + 2 chapatis | Fish Tikka + mixed salad |
| Fri | Paratha + butter yogurt | Khichdi + pickle | Paneer Tikka + naan |
| Sat | Masala Oats (spiced) | Mutton Rogan Jamun + roti | Lentil Soup + bhature |
| Sun | Fresh fruit + boiled egg | Bhindi Masala + 2 phulkas | Chicken Saag + steamed rice |

*Feel free to swap any side dish or adjust spices to taste. Enjoy the variety!*


In [ ]:
## Traces
import mlflow
import asyncio
from agents import Agent, Runner

mlflow.openai.autolog()

# Optional: Set a tracking URI and an experiment
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("OpenAI Agent")

<Experiment: artifact_location='mlflow-artifacts:/615119549503924654', creation_time=1756996413561, experiment_id='615119549503924654', last_update_time=1756996413561, lifecycle_stage='active', name='OpenAI Agent', tags={}>

In [ ]:
# Enable auto tracing for OpenAI Agents SDK
mlflow.openai.autolog()


@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny."


agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],
    model=model
)

result = await Runner.run(agent, input="What's the weather in Tokyo?")
print(result.final_output)

The weather in Tokyo is sunny.


Trace(trace_id=tr-08f6a7426f866f388ab068a3bc31f070)

In [ ]:
# === 🧠 AGENT: Emoji Generator ===
emoji_agent = Agent(
    name="emoji_agent",
    instructions=(
        "You're an emoji artist. You turn summarized news into expressive emoji + text sequences. "
        "Be creative, thematic, and intuitive — like visual headlines that combine words and emojis for clarity and impact. "
        "Use emojis to replace or enhance key nouns, verbs, places, or themes. Always return a short sentence or phrase with integrated emojis."
    ),
    model=model
)

emoji_tool = emoji_agent.as_tool(
    tool_name="emoji_generator",
    tool_description="Generates creative emoji + text representations from short summarized news text."
)

# === ✏️ AGENT: Summarizer ===
summarizer = Agent(
    name="summarizer",
    instructions=(
        "You're a news summarizer. Your job is to shorten long news into crisp sentences. "
        "Avoid emojis — keep it simple, and informative."
    ),
    model=model
)

summarizer_tool = summarizer.as_tool(
    tool_name="text_summarizer",
    tool_description="Summarizes long text into short, emoji-ready summaries."
)

# === 🎬 AGENT: Orchestrator ===
emoji_headline_generator = Agent(
    name="emoji_headlines",
    instructions=(
        "You're an assistant that turns full news stories into emoji-enhanced headlines. "
        "Use the available tools: first summarize with the summarizer, then generate a headline with emojis using the emoji generator. "
        "Do not generate emojis directly — always use the tools. Final output should be a compact sentence combining emojis and text where suitable."
    ),
    tools=[summarizer_tool, emoji_tool],
    model=model
)

# === 🧪 TEST DATA ===
news_sample = """
OpenAI has announced that ChatGPT’s Projects feature is now available to Free users, expanding access beyond paid tiers. The update also includes larger file uploads, more customisation options, and new memory controls. According to OpenAI, Free users can now upload up to five files per project, while Plus subscribers can upload 25 and Pro, Business, and Enterprise customers up to 40.  Users will also be able to select colours and icons for projects, as well as manage memory on a project-only basis. The rollout is live on web and Android, with iOS support expected in the coming days. In a separate development, OpenAI confirmed that the team behind Alex, an AI-powered coding assistant for Xcode, has joined its Codex team. Daniel Edrisian, co-founder of Alex, said in a pos
"""

# === 🚀 RUN WORKFLOW ===
emoji_response = await Runner.run(emoji_headline_generator, news_sample)
emoji_output = emoji_response.final_output

# === 📤 DISPLAY ===
print("🗞️  Original News:")
print(news_sample)
print("\n✨ Emoji Headline:")
display(Markdown(emoji_output))


🗞️  Original News:

OpenAI has announced that ChatGPT’s Projects feature is now available to Free users, expanding access beyond paid tiers. The update also includes larger file uploads, more customisation options, and new memory controls. According to OpenAI, Free users can now upload up to five files per project, while Plus subscribers can upload 25 and Pro, Business, and Enterprise customers up to 40.  Users will also be able to select colours and icons for projects, as well as manage memory on a project-only basis. The rollout is live on web and Android, with iOS support expected in the coming days. In a separate development, OpenAI confirmed that the team behind Alex, an AI-powered coding assistant for Xcode, has joined its Codex team. Daniel Edrisian, co-founder of Alex, said in a pos


✨ Emoji Headline:


OpenAI frees Projects: 📁 uploads ↑ (5 → 25 → 40), 🎨 icons, 💾 per‑project memory. Live on 🖥️+📱, iOS next. Xcode AI Alex joins the Codex team 🤝🧠

Trace(trace_id=tr-da212cbbfee56a04b4924933140dc01a)

In [ ]:

# Define a simple multi-agent workflow
hindi_agent = Agent(
    name="Hindi agent",
    instructions="You only speak Hindi.",
    model=model
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
    model=model
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[hindi_agent, english_agent],
    model=model
)


result = await Runner.run(triage_agent, input="एआई एजेंटों के बड़े में तुम क्या जानते हो?")
display(Markdown(result.final_output))


एआई एजेंटों के बड़े मॉडल (जैसे GPT‑4, Claude, Gemini आदि) के बारे में मैं मुख्यतः ये बता सकता हूँ:

1. **आर्किटेक्चर और आकार**  
   - अधिकांश बड़े एआई मॉडल ट्रांसफॉर्मर‑आधारित होते हैं।  
   - पैरामीटर की संख्या अरबों (10‑100 B) से लेकर खरबों (1 T या उससे अधिक) तक हो सकती है।  
   - यह बड़ा आकार उन्हें जटिल पैटर्न, सहसंबंध व संदर्भ समझने में सक्षम बनाता है।

2. **प्रशिक्षण डेटा**  
   - बड़े पैमाने पर इंटरनेट से लिए गए टेक्स्ट, PDFs, वेबसाइट्स, सोशल मीडिया, कोड रिपॉज़िटरी आदि पर प्रशिक्षित।  
   - डेटा में 2021‑2023 की ताज़ा जानकारी, तथा उससे परे की भविष्यवाणी के लिए पूर्वानुमान आधारित मॉडलिंग का उपयोग भी होता है।

3. **क्षमताएँ**  
   - बहुभाषी संवाद और अनुवाद।  
   - सार‑संक्षेप, रचनात्मक लेखन, कोड जनरेशन, डेटा विश्लेषण।  
   - कुछ मॉडलों में “फ़ाइन‑ट्यूनिंग” या “री‑फाइन‑ट्यूनिंग” के द्वारा विशिष्ट टास्क, जैसे इमेज‑टू‑टेक्स्ट या टेबल पढ़ना, में दक्षता बढ़ाई जा सकती है।  
   - प्रॉम्प्ट इंजीनियरिंग के माध्यम से कार्य को “डायलॉग बॉक्स” से परे परिष्कृत किया जा सकता है।

4. **सीमाएँ**  
   - **तथ्यात्मकता**: मॉडल को “ज्ञान” से अधिक “पैटर्न” सीखता है, अतः झूठी या काल्पनिक जानकारी देने की सम्भावना रहती है।  
   - **पूर्वाग्रह**: प्रशिक्षण डेटा में मौजूद लैंगिक, सांस्कृतिक, वंशीय पूर्वाग्रह भी मॉडल में प्रकट हो सकते हैं।  
   - **स्रोत अनिश्चितता**: किसी तथ्य का स्रोत (कहाँ से सीखा) मॉडल स्वयं नहीं बताता, इसलिए सत्यापन हमेशा ज़रूरी है।  
   - **संसाधन**: बड़े मॉडल का प्रशिक्षण व inference दोनों ही भारी कम्प्यूटिंग व ऊर्जा की माँग करते हैं, जिससे पर्यावरणीय चिंता भी उठती है।  

5. **उपयोग के क्षेत्र**  
   - ग्राहक सहायता चैटबॉट, लेखन सहायक, कोड‑राइटिंग टूल, शोध सहायता, शिक्षा, हेल्थ‑केयर सलाह, सिमुलेशन‑आधारित प्रशिक्षण, रचनात्मक कला।  
   - कंपनियाँ अक्सर इसे अपने उत्पादों में embed करती हैं, जैसे Gmail के compose सुझाव, या GitHub Copilot।  

6. **सुरक्षा और नैतिकता**  
   - OpenAI और अन्य संगठनों ने “उपयोग नीति”, “सुरक्षा ट्यूनिंग” व “प्रॉम्प्ट‑ब्लॉकिंग” जैसी तकनीकें अपनाई हैं।  
   - मॉडल को संवेदनशील डेटा पर फाइन‑ट्यूनिंग में डेटा गोपनीयता तथा HIPAA, GDPR जैसी नियमों का पालन करना आवश्यक है।  

7. **भविष्य की दिशा**  
   - **मॉडेल फ्यूज़न**: GPT‑जैसे टेक्स्ट मॉडल से इमेज, ऑडियो, वीडियो को एकीकृत करके मल्टी‑मॉडल अनुभव।  
   - **फ़ेडरेटेड लर्निंग**: डेटा को केंद्रीकृत न करके उपयोगकर्ता के डिवाइस पर ही मॉडल सिखाने का प्रयास।  
   - **सुपर‑कम्पैटिबिलिटी**: बड़े एआई को कम‑पावर हार्डवेयर (जैसे मोबाइल, एज डिवाइस) पर चलाने के लिए नॉलेज डिस्टिलेशन व pruning।  
   - **इंटेलिजेंट एजेंट**: अपने कार्य‑प्रवाह में स्वचालित निर्णय लेने के लिए योजनाकार, योजना‑निर्माता, व री‑लर्निंग लूप शामिल।  

सारांश में, बड़े एआई एजेंट्स बहुक्षेत्रीय ज्ञान‑संचालकों के रूप में उभर रहे हैं, परंतु इनके उपयोग में सटीकता, पूर्वाग्रह, गोपनीयता और पर्यावरणीय पहलुओं पर सतर्कता अवश्य रखनी चाहिए।

Trace(trace_id=tr-8688f44eff649ec5b4fefd5a8b7c7c4c)

In [ ]:

# === 📝 User prompt ===
prompt = "Who is the prime minister of Pakistan?"

# === 📦 Input Guardrail Pydantic Model ===
class PersonalIdentifier(BaseModel):
    is_personal_info_in_message: bool
    name: str = ""  # Use str to match agent instructions

# === 🛡️ Agent to detect PII in user prompt ===
input_guardrail_agent = Agent(
    name="PID Checker",
    instructions=(
        "Analyze the input message for personal identity information (PII) such as names, phone numbers, or addresses. "
        "Return a JSON object with 'is_personal_info_in_message' set to true if PII is found, false otherwise. "
        "If a name is detected, set 'name' to the full name (e.g., 'John Smith'); otherwise, set it to an empty string. "
        "Ensure the response is valid JSON in this exact format:\n"
        "```json\n"
        "{\n  \"is_personal_info_in_message\": true,\n  \"name\": \"John Smith\"\n}\n"
        "```\n"
        "or\n"
        "```json\n"
        "{\n  \"is_personal_info_in_message\": false,\n  \"name\": \"\"\n}\n"
        "```\n"
        "Examples:\n"
        "- Input: 'Can you tell me about John Smith?' → Output: {\"is_personal_info_in_message\": true, \"name\": \"John Smith\"}\n"
        "- Input: 'What is the capital of France?' → Output: {\"is_personal_info_in_message\": false, \"name\": \"\"}\n"
        "Return only the JSON object, nothing else."
    ),
    output_type=PersonalIdentifier,
    model=model
)

# === 🛡️ Input Guardrail function ===
@input_guardrail
async def guardrail_against_pid(ctx, agent, message):
    try:
        result = await Runner.run(input_guardrail_agent, message, context=ctx.context, max_turns=5)
        found_pid = result.final_output
        if found_pid:
            print(f"Input Guardrail Result: {found_pid}")
            print(f"Detected Name: {found_pid.name}")
        else:
            print("Input Guardrail Result: None (no valid output)")
        return GuardrailFunctionOutput(
            output_info={"found_name": found_pid.name if found_pid else "", "raw_output": result.raw_output if result else None},
            tripwire_triggered=found_pid.is_personal_info_in_message if found_pid else False
        )
    except Exception as e:
        print(f"Error in input guardrail: {e}")
        return GuardrailFunctionOutput(
            output_info={"found_name": "", "error": str(e)},
            tripwire_triggered=False
        )

# === 📦 Output Guardrail Pydantic Model ===
class ToxicityOutput(BaseModel):
    is_toxic: bool
    toxic_words: List[str] = []

# === 🧠 Agent to detect toxic output using LLM ===
toxicity_checker_agent = Agent(
    name="Toxicity Checker",
    instructions=(
        "Analyze the assistant's message for toxic, offensive, or inappropriate language. "
        "Return a JSON object with 'is_toxic' set to true if toxic content is found, false otherwise. "
        "List any toxic words in 'toxic_words' as an array; use an empty array if none are found. "
        "Ensure the response is valid JSON in this format:\n"
        "```json\n"
        "{\n  \"is_toxic\": true,\n  \"toxic_words\": [\"word1\", \"word2\"]\n}\n"
        "```\n"
        "or\n"
        "```json\n"
        "{\n  \"is_toxic\": false,\n  \"toxic_words\": []\n}\n"
        "```\n"
        "Return only the JSON object."
    ),
    output_type=ToxicityOutput,
    model=model
)

# === 🛡️ Output Guardrail function ===
@output_guardrail
async def guardrail_against_toxic_output(ctx, agent, response):
    try:
        result = await Runner.run(toxicity_checker_agent, response, context=ctx.context)
        toxicity_info = result.final_output
        return GuardrailFunctionOutput(
            output_info={"toxicity": toxicity_info, "raw_output": result.raw_output if result else None},
            tripwire_triggered=toxicity_info.is_toxic if toxicity_info else False
        )
    except Exception as e:
        print(f"Error in output guardrail: {e}")
        return GuardrailFunctionOutput(
            output_info={"toxicity": None, "error": str(e)},
            tripwire_triggered=False
        )

# === 💬 Q&A Agent ===
qa_agent = Agent(
    name="Q&A",
    instructions="You're a helpful and respectful assistant. Answer user questions accurately and professionally.",
    model=model,
    input_guardrails=[guardrail_against_pid],
    output_guardrails=[guardrail_against_toxic_output],
)

# === 🏃‍♂️ Run the agent with guardrails ===

try:
    result = await Runner.run(qa_agent, prompt)
    # === 📤 Display ===
    print("📝 User Prompt:")
    print(prompt)
    print("\n✅ Final Output:")
    print(result.final_output)
    print("\n🛡️ Input Guardrail Info:")
    print(result.guardrail_input_output_info)
    print("\n🛡️ Output Guardrail Info:")
    print(result.guardrail_output_output_info)
except Exception as e:
    print(f"Error running agent: {e}")


Error in input guardrail: Max turns (5) exceeded
Error in output guardrail: Max turns (10) exceeded
📝 User Prompt:
Who is the prime minister of Pakistan?

✅ Final Output:
**Prime Minister of Pakistan (as of April 2025)**

- **Shehbaz Sharif**  
  • In office since 13 August 2022 after the PML‑N coalition secured the majority.  
  • He survived the 2023 no‑confidence motion and remained in the post through the rest of the parliamentary term.

> *Note:* Pakistan’s parliamentary terms are five years, and any upcoming general election (currently scheduled for early to mid‑2025) could change the holder of the office. Always check a reliable, up‑to‑date source for the most current information.

🛡️ Input Guardrail Info:
Error running agent: 'RunResult' object has no attribute 'guardrail_input_output_info'


Trace(trace_id=tr-bbff6626d2cbae6a5718cc961b245caf)